# Demand prediction baseline solution

Ukhlinova Liza 492

### Отчет:

(a) Задача состояла в прогнозировании спроса товаров на первые три недели 2015 года по данным за 2012-2014 года

(b) Итоговое решение: исходный датасет преобразовывался с учетом сдвига признаков. Так как в строке напротив y (который был использован как таргет) стояли признаки, которые соответсвовали времени "week - shift", я переставила признаки на строки, с таким же item_id и соответсвующей неделей. В итоге у меня получились строки с ['y', 'year', 'week', 'item_id', 'f1'...'f60'], где признаки соответсвуют этой неделе и году. В итоге исходная тренировочная выборка сильно уменьшилась в размерах. Аналогичное преобразование я сделала с тестовой выборкой, но там пришлось разделить данные на 2 датасета - строки, для которых есть признаки с соответствующей датой, и строки без правильных признаков. В качестве модели я использовала XGBRegressor. Первую часть я предсказывала по модели, обученной на преобразованном train, а вторую часть - на исходном датасете. В конце решения сшивались в одно, большинство из них были обчуены на корректной выборке. 

(c) Сначала я пыталась просто поменять некоторые параметры в коде GradientBoostingRegressor - какую часть выборки использовать, n_estimators, высоту дерева и т.д. С таким кодом удалось достичь SMAPE около 50. Затем я решила брать не все строки - брала только каждую вторую, затем каждую пятую - такая стратегия привела к 36 очкам. Но с увеличением прореженность строк, мой score падал. Кодирование item_id также не дало хороших результатов.  Загрузив признаки в excel стало заметно, что они сильно скореллированы и скорее всего хорошо бы большую часть из них убрать. Я пробовала метод corr() и выбирала 5-10 наименее скореллированных признаков. Но это не помогло. В ход пошли методы SelectKBest, RFE - recursive feature elimination и оценка feature_importance для отбора фич. Но результат получался всегда хуже тупого прореживания строк. Было замечено, что вторая половина столбцов точно такая же, как первая, поэтому я обрезала датасет и попробовала запустить XGBoost на нем. Поварьировав параметры удалось дойти до score 25. Когда я поняла, что shift не просто так дан в таблице, я постепенно пришла к итоговому решению.

(e) Для оценки качества я написала простую функцию SMAPE и проверяла результаты на кросс валидации, обучаясь на 80% train и проверяю на остальном. Результат был очень близок к тому, что выдавал kaggle. Также в самом начале испольщовала mean_squared_error, но результаты не очень соответствовали итогам на kaggle. Минимизация приводила к самому большому SMAPE.

Дополнительные вопросы:
    
1. Для sample_submission могли взять среднее значение y из тестовой выборки (или из тренировочной) и продублировать его 2016 раз. 

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [3]:
def smape(pred, y):
        return float(200) / len(y) * sum(abs(pred - y) / (abs(y) + abs(pred)))

In [185]:
frac = 1.0
train = train.sample(frac=frac, random_state=81)

### Закодируем item_id

In [186]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train.item_id)
train['item_id'] = le.transform(train.item_id)
train.head()

,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
31868,157312,578661,2013,47,2,217,325401.0,462269.0,423886.0,303062.0,...,322328.0,380034.0,417557.0,441174.0,364465.0,339045.0,383666.0,368462.0,544448.0,192131.0
70199,343901,1027927,2014,52,2,72,593858.0,651118.0,625255.0,552061.0,...,511137.0,505528.0,517679.0,616331.0,384994.0,499037.0,471680.0,515997.0,459638.0,460340.0
5686,27784,958861,2013,8,1,36,1136500.0,1099795.0,817529.0,814748.0,...,930002.0,1072060.0,1538955.0,390422.0,315140.0,1107287.0,871260.0,1120653.0,552481.0,545769.0
5684,27782,71500,2013,8,1,34,82340.0,121672.0,59706.0,75390.0,...,42320.0,56900.0,194110.0,24460.0,29700.0,41530.0,43380.0,145250.0,29090.0,36640.0
34864,170980,9595,2013,52,3,124,4510.0,4530.0,5460.0,4542.0,...,5690.0,3850.0,5274.0,5749.0,5020.0,2525.0,3820.0,3990.0,5130.0,3420.0


In [187]:
y = train['y']
X = train.drop(['Num','y'], axis=1)

### Попробуем отобрать признаки, использую RFE

In [188]:
%%time
from sklearn.feature_selection import RFE

rfe = RFE(model, 5) 
selector = rfe.fit(X,y)
selector.ranking_

CPU times: user 12 µs, sys: 11 µs, total: 23 µs
Wall time: 26 µs


In [189]:
for i in range(len(selector.ranking_)):
    if (selector.ranking_[i] == 1):
        print(i)
print(selector.ranking_)

In [190]:
X_RFE_10 = X[['year','f4','f11','f23','f27', 'f28','f29','f30','f39', 'f56']]
test_RFE_10 = test[['year','f4','f11','f23','f27', 'f28','f29','f30','f39', 'f56']]
X_RFE_10.shape

(72457, 10)

In [191]:
X_RFE_5 = X[['year','f11','f23','f28','f30']]
test_RFE_5 = test[['year','f11','f23','f28','f30']]
X_RFE_5.shape

(72457, 5)

In [205]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=500, max_depth=5, random_state=81)
model.fit(X_RFE_10, y)

preds = model.predict(test_RFE_10)

print (len(preds))
print (len(sample_submission))

2016
2016
CPU times: user 55.3 s, sys: 771 ms, total: 56.1 s
Wall time: 57.6 s


Не очень помогло, SMAPE был порядка 40

### Теперь будем брать не все строчки для обучения

In [44]:
X_droped = train[train.Num % 5 == 0]
y_droped = X_droped['y']
X_droped = X_droped.drop(['Num','y'], axis=1)
print (len(X_droped), len(y_droped))

14496 14496


In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=500, max_depth=5, random_state=81)
model.fit(X_droped, y_droped)

preds = model.predict(test)

print (len(preds))
print (len(sample_submission))

Результат был хороший - SMAPE = 36 

### Попробуем использовать shift и переделаем нашу выборку - поставим признаки на правильные места

In [2]:
train.head()

,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,0,123438,2012,52,1,20442076,4915.0,38056.0,40185.0,45733.0,...,39423.0,41765.0,52590.0,31452.0,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0
1,1,58410,2012,52,1,20441997,2230.0,18817.0,20110.0,26368.0,...,22830.0,25230.0,27850.0,21390.0,27090.0,23170.0,29705.0,19140.0,22055.0,23200.0
2,2,163930,2012,52,1,20441990,5695.0,47480.0,47619.0,89708.0,...,14930.0,44290.0,46412.0,29320.0,21140.0,28406.0,65056.0,31886.0,48750.0,36520.0
3,3,53902,2012,52,1,20441989,1995.0,17146.0,20066.0,27070.0,...,15120.0,12480.0,19780.0,7990.0,8230.0,10650.0,21920.0,13040.0,9780.0,9630.0
4,4,105970,2012,52,1,20441988,6515.0,49262.0,50045.0,95167.0,...,18872.0,19328.0,37168.0,13570.0,19760.0,20208.0,34745.0,18442.0,24700.0,21793.0


In [32]:
X_new = pd.read_csv("train.tsv")
X_new['features_week'] = X_new['week'] - X_new['shift']
X_new['features_year'] = X_new['year']

for i in range(len(X_new)):
    if (X_new['features_week'][i] <= 0):
        if (X_new.loc[i,'features_year'] == 2015):
            X_new.loc[i,'features_week'] = 53 + X_new.loc[i,'features_week']
        else:
            X_new.loc[i,'features_week'] = 52 + X_new.loc[i,'features_week']
        X_new.loc[i,'features_year'] -= 1

In [33]:
frac = 0.5
X_new = X_new.sample(frac=frac, random_state=81)
#X_new = X_new[:20000]
len(X_new)
X_new.head()

,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f53,f54,f55,f56,f57,f58,f59,f60,features_week,features_year
31868,157312,578661,2013,47,2,20448083,325401.0,462269.0,423886.0,303062.0,...,417557.0,441174.0,364465.0,339045.0,383666.0,368462.0,544448.0,192131.0,45.0,2013.0
70199,343901,1027927,2014,52,2,20438694,593858.0,651118.0,625255.0,552061.0,...,517679.0,616331.0,384994.0,499037.0,471680.0,515997.0,459638.0,460340.0,50.0,2014.0
5686,27784,958861,2013,8,1,20438539,1136500.0,1099795.0,817529.0,814748.0,...,1538955.0,390422.0,315140.0,1107287.0,871260.0,1120653.0,552481.0,545769.0,7.0,2013.0
5684,27782,71500,2013,8,1,20438537,82340.0,121672.0,59706.0,75390.0,...,194110.0,24460.0,29700.0,41530.0,43380.0,145250.0,29090.0,36640.0,7.0,2013.0
34864,170980,9595,2013,52,3,20440443,4510.0,4530.0,5460.0,4542.0,...,5274.0,5749.0,5020.0,2525.0,3820.0,3990.0,5130.0,3420.0,49.0,2013.0


In [34]:
%%time
for index, row in X_new.iterrows():
    flag = False
    items = X_new[X_new['item_id'] == row['item_id']]
    for index_, row_ in items.iterrows():
        if (row['year'] == row_['features_year'] and
            row['week'] == row_['features_week']):
            X_new.loc[index,'f1':'f60'] = row_['f1':'f60'] 
            flag = True
            break
    if (flag == False):
        X_new.drop(index, inplace=True)
        
y_new = X_new['y']
X_new = X_new.drop(['Num','y','shift','features_week','features_year'],axis=1)

CPU times: user 11min 52s, sys: 26.5 s, total: 12min 19s
Wall time: 12min 26s


In [35]:
print(len(X_new))
X_new.head()

29420


,year,week,item_id,f1,f2,f3,f4,f5,f6,f7,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
31868,2013,47,20448083,423886.0,303062.0,394460.0,397174.0,592734.0,330116.0,294559.0,...,417557.0,441174.0,364465.0,339045.0,383666.0,368462.0,544448.0,192131.0,355802.0,359417.0
70199,2014,52,20438694,625255.0,552061.0,645081.0,681047.0,607230.0,597449.0,587464.0,...,517679.0,616331.0,384994.0,499037.0,471680.0,515997.0,459638.0,460340.0,509050.0,638464.0
5686,2013,8,20438539,1099795.0,817529.0,814748.0,919480.0,1372384.0,642415.0,713666.0,...,1072060.0,1538955.0,390422.0,315140.0,1107287.0,871260.0,1120653.0,552481.0,545769.0,595566.0
5684,2013,8,20438537,121672.0,59706.0,75390.0,88745.0,207960.0,43760.0,53771.0,...,56900.0,194110.0,24460.0,29700.0,41530.0,43380.0,145250.0,29090.0,36640.0,44410.0
34864,2013,52,20440443,4542.0,2680.0,6355.0,5670.0,5390.0,4395.0,6620.0,...,5749.0,5020.0,2525.0,3820.0,3990.0,5130.0,3420.0,4000.0,5030.0,5960.0


In [36]:
test_new = pd.read_csv("test.tsv")
test_new['features_week'] = test_new['week'] - test_new['shift']
test_new['features_year'] = test_new['year']

for i in range(len(test_new)):
    if (test_new['features_week'][i] <= 0):
        if (test_new.loc[i,'features_year'] == 2015):
            test_new.loc[i,'features_week'] = 53 + test_new.loc[i,'features_week']
        else:
            test_new.loc[i,'features_week'] = 52 + test_new.loc[i,'features_week']
        test_new.loc[i,'features_year'] -= 1
test_noanswer = test_new.loc[:1]

In [37]:
print(len(test_new))

2016


In [38]:
%%time
indexes = []

for index, row in test_new.iterrows():
    flag = False
    items = test_new[test_new['item_id'] == row['item_id']]
    for index_, row_ in items.iterrows():
        if (row['year'] == row_['features_year'] and
            row['week'] == row_['features_week']):
            test_new.loc[index,'f1':'f60'] = row_['f1':'f60'] 
            indexes.append(index)
            flag = True
            break
    if (flag == False):
        test_new.drop(index, inplace=True)
        
test_new = test_new.drop(['Num','shift','features_week','features_year'],axis=1)     

CPU times: user 21.5 s, sys: 232 ms, total: 21.8 s
Wall time: 22.4 s


In [39]:
print(len(test_new))

1335


#### Для данных без нормальных фич:

In [46]:
X_usual = pd.read_csv("train.tsv")
X_usual = X_usual.sample(frac=0.5, random_state=81)

y_usual = X_usual['y']
X_usual = X_usual.drop(['Num','y'],axis=1)
test_usual = pd.read_csv("test.tsv")
test_usual = test_usual.drop(['Num'],axis=1)
print(len(X_usual))

36228


In [41]:
%%time
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_new, y_new, 
                                                                     test_size=0.20, 
                                                                     random_state=1)

model_xgb_2.fit(X_train, y_train)
y_pred = model_xgb_2.predict(X_test)
print(smape(y_pred,y_test))

11.0823780793
CPU times: user 5min 4s, sys: 1.51 s, total: 5min 5s
Wall time: 5min 7s


Отличный результат! Но это только для данных, у которых есть правильные признаки на тестовой выборке. С остальными данными будем использовать всю train выборку для обучения.

In [47]:
%%time
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_usual, y_usual, 
                                                                     test_size=0.20, 
                                                                     random_state=1)

model_xgb_2.fit(X_train, y_train)
y_pred = model_xgb_2.predict(X_test)
print(smape(y_pred,y_test))

23.1243867311
CPU times: user 7min 24s, sys: 3.26 s, total: 7min 28s
Wall time: 7min 34s


Вроде бы тоже неплохо! Запустим по отдельности на каждую часть выборки и сгруппируем

In [54]:
%%time
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor

model_usual = GradientBoostingRegressor(n_estimators=500, max_depth=5, random_state=81)
model_xgb_2 = xgb.XGBRegressor(n_estimators=820, max_depth=19, learning_rate=0.05)
model_xgb_2.fit(X_usual, y_usual)

CPU times: user 8min 59s, sys: 2.25 s, total: 9min 1s
Wall time: 9min 4s


In [55]:
%%time
preds_usual = model_xgb_2.predict(test_usual)

print (len(preds_usual))
print (len(preds_usual[preds_usual < 0]))

2016
1
CPU times: user 1.7 s, sys: 10.2 ms, total: 1.71 s
Wall time: 1.72 s


### Для данных с нормальными фичами:

In [56]:
%%time

model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=100)

CPU times: user 16 µs, sys: 1e+03 ns, total: 17 µs
Wall time: 19.8 µs


In [57]:
%%time

model_xgb_2.fit(X_new, y_new)
preds_new = model_xgb_2.predict(test_new)

print (len(preds_new))
print (len(preds_new[preds_new < 0]))
print(preds_new)

1335
3
[   1399.79431152   25985.75390625  304728.96875    ...,   26049.55859375
     369.25335693    5889.49316406]
CPU times: user 6min 16s, sys: 1.98 s, total: 6min 18s
Wall time: 6min 21s


In [60]:
print((np.array(indexes)))

[   0    1    2 ..., 1573 1574 1575]


In [61]:
for num, k in enumerate(indexes):
    preds_usual[k] = preds_new[num]

print(len(preds_usual))
preds_usual

2016


array([   1399.79431152,   25985.75390625,  304728.96875   , ...,
         21283.13671875,     410.88430786,    4284.0546875 ], dtype=float32)

In [62]:
sample_submission['y'] = preds_usual
sample_submission.head(5)
print (len(sample_submission[sample_submission['y'] < 0]))
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

4


In [65]:
preds_usual.mean()

149144.67

Этот результат дал SMAPE = 21 на kaggle.